### ERA5 Data Extraction for Census Places

In [2]:
import numpy as np
import geopandas as gpd
import gdown
import ee
import google
import os
import geemap
import warnings
warnings.filterwarnings('ignore')

from tobler.util import h3fy


In [3]:
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1Adeu5BWGz_ArfFT0RryA5HY2wAmDpnbrsDEagyRCLZUD_OoiBz48LFP21AY

Successfully saved authorization token.


### Get ERA5 hourly data by h3 hexagons, resolution 5

In [4]:
era5 = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")
hourly2022 = era5.filterDate('2022-01-01', '2022-01-02').select('temperature_2m')

In [5]:
def get_ee_feature(geom):
    x,y = geom.exterior.coords.xy
    coords = np.dstack((x,y)).tolist()
    g = ee.Geometry.Polygon(coords)
    return ee.Feature(g)

Data source: https://catalog.data.gov/dataset/tiger-line-shapefile-2019-state-alaska-current-alaska-native-regional-corporation-anrc-state-ba

In [34]:
features.size().getInfo()

7287

In [11]:
# ak shapefile
geodf_alaska = gpd.read_file('../Data/tl_2019_02_anrc/tl_2019_02_anrc.shp')
geodf_alaska = geodf_alaska.to_crs(epsg=3338) 
# to hexagons
hexgrid_ak = h3fy(geodf_alaska.buffer(0.01), resolution=5)
# to feature collection
features = ee.FeatureCollection([get_ee_feature(x[1].geometry) for x in hexgrid_ak.iterrows()])

In [12]:
map1 = geemap.Map()

rectangle_left = ee.Geometry.Rectangle([-200, 10, -150, 80])
rectangle_right = ee.Geometry.Rectangle([-150, 10, -100, 80])

map1.addLayer(rectangle1, {'color': "yellow"}, 'AK Left')
map1.addLayer(rectangle2, {'color': 'blue'}, 'AK Right')


map1

2

In [90]:
filtered_features_AK_left = features.filterBounds(rectangle_left)
filtered_features_AK_right = features.filterBounds(rectangle_right)

left_size = filtered_features_AK_left.size().getInfo()
right_size = filtered_features_AK_right.size().getInfo()
print(left_size, right_size, left_size + right_size)

4260 4221 8481


In [91]:
# reducer 
def reduceByFeatureLeft(image):
  return image.reduceRegions(collection = filtered_features_AK_left,
                             reducer = ee.Reducer.mean().setOutputs(['avg_air_temp']),
                            scale = 11132)
def reduceByFeatureRight(image):
  return image.reduceRegions(collection = filtered_features_AK_right,
                             reducer = ee.Reducer.mean().setOutputs(['avg_air_temp']),
                            scale = 11132)

In [92]:
hourlyClippedLeft = hourly2022.map(lambda image: image.clipToCollection(filtered_features_AK_left))

hexExportLeft = hourlyClippedLeft.map(reduceByFeatureLeft).flatten()

In [93]:
hourlyClippedRight = hourly2022.map(lambda image: image.clipToCollection(filtered_features_AK_right))

hexExportRight = hourlyClippedRight.map(reduceByFeatureRight).flatten()

In [94]:
task = ee.batch.Export.table.toDrive(**{
  'collection': hexExportRight,
  'description': 'Hexagons Average Temp by Hourly 2022 Try 6 Right',
  'folder': 'EarthEngine',
  'selectors': ['system:index','NAME', 'FIPS', 'avg_air_temp'], 
  'fileFormat': 'CSV'
})
task.start()

In [95]:
task = ee.batch.Export.table.toDrive(**{
  'collection': hexExportLeft,
  'description': 'Hexagons Average Temp by Hourly 2022 Try 6 Left',
  'folder': 'EarthEngine',
  'selectors': ['system:index','NAME', 'FIPS', 'avg_air_temp'], 
  'fileFormat': 'CSV'
})
task.start()

In [5]:
# download the result from Google drive (later change to Github submodule)
url = "https://drive.google.com/file/d/1T1vRS8OU1S89sfaoDCynN9BxAKRlGRPO/view?usp=drive_link"
output_path = "../Data/"
gdown.download(url, output_path, quiet=False,fuzzy=True)

Downloading...
From (uriginal): https://drive.google.com/uc?id=1T1vRS8OU1S89sfaoDCynN9BxAKRlGRPO
From (redirected): https://drive.google.com/uc?id=1T1vRS8OU1S89sfaoDCynN9BxAKRlGRPO&confirm=t&uuid=7927446a-026c-4a3f-a776-b0f4f8d038e1
To: /Users/brianleung/Documents/Python-Projects/DSSG2023-Heating-Loads/Data/Places Average Temp by Hourly 2022.csv
100%|██████████| 226M/226M [00:10<00:00, 21.5MB/s] 


'../Data/Places Average Temp by Hourly 2022.csv'